In [38]:
!pwd

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
pwd: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory


In [36]:
!ls|

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [39]:
!git status

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: Unable to read current working directory: No such file or directory


In [43]:
%cd /

/


In [44]:
%cd /content

/content


In [45]:
!ls

drive  sample_data


In [46]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [47]:
%cd predicting-student-scores

/content/drive/MyDrive/predicting-student-scores


In [31]:
!git add .

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: Unable to read current working directory: No such file or directory


In [30]:
!git add .
!git commit -m "Add baseline model"
!git push -u origin baseline

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: Unable to read current working directory: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: Unable to read current working directory: No such file or directory
shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
fatal: Unable to read current working directory: No such file or directory


In [28]:
!git push -u origin baseline

Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
remote: 
remote: Create a pull request for 'baseline' on GitHub by visiting:
remote:      https://github.com/Xeoyeon/predicting-student-scores/pull/new/baseline
remote: 
To https://github.com/Xeoyeon/predicting-student-scores.git
 * [new branch]      baseline -> baseline
Branch 'baseline' set up to track remote branch 'baseline' from 'origin'.


In [9]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error

In [10]:
# load dataset
path='/content/drive/MyDrive/data_analysis/kaggle_datasets/student-data/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
submission = pd.read_csv(path+'sample_submission.csv')

In [11]:
# EDA
train.head()

,id,age,gender,course,study_hours,class_attendance,internet_access,sleep_hours,sleep_quality,study_method,facility_rating,exam_difficulty,exam_score
0,0,21,female,b.sc,7.91,98.8,no,4.9,average,online videos,low,easy,78.3
1,1,18,other,diploma,4.95,94.8,yes,4.7,poor,self-study,medium,moderate,46.7
2,2,20,female,b.sc,4.68,92.6,yes,5.8,poor,coaching,high,moderate,99.0
3,3,19,male,b.sc,2.00,49.5,yes,8.3,average,group study,high,moderate,63.9
4,4,23,male,bca,7.65,86.9,yes,9.6,good,self-study,high,easy,100.0


In [12]:
train.isnull().sum() #결측값 개수 세기

,0
id,0
age,0
gender,0
course,0
study_hours,0
class_attendance,0
internet_access,0
sleep_hours,0
sleep_quality,0
study_method,0


In [13]:
#preprocessing
le = LabelEncoder()

object_cols = train.select_dtypes(include=['object']).columns #object 타입 컬럼만 고르기

for col in object_cols:
  train_values = set(train[col].astype(str).unique())
  test_values = set(test[col].astype(str).unique())

  all_labels = list(train_values | test_values) #합집합임 -> 중복 제거 된다.
  print(all_labels)

  le.fit(all_labels)
  train[col]=le.transform(train[col].astype(str))
  if col in test.columns:
    test[col]=le.transform(test[col].astype(str))

['female', 'other', 'male']
['bba', 'bca', 'b.sc', 'b.tech', 'b.com', 'ba', 'diploma']
['yes', 'no']
['good', 'average', 'poor']
['coaching', 'group study', 'self-study', 'online videos', 'mixed']
['medium', 'low', 'high']
['hard', 'easy', 'moderate']


In [14]:
display(train[object_cols].head())

,gender,course,internet_access,sleep_quality,study_method,facility_rating,exam_difficulty
0,0,1,0,0,3,1,0
1,2,6,1,2,4,2,2
2,0,1,1,2,0,0,2
3,1,1,1,0,1,0,2
4,1,5,1,1,4,0,0


In [15]:
# train, test data
target_col = 'exam_score' #예측해야 할 값. 정답
X = train.drop(columns=['id',target_col])
y = train[target_col]
X_test = test.drop(columns=['id'])

In [16]:
# train
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(n_estimators=100, learning_rate=0.05) #default : max_depth=6
model.fit(X_train,y_train)

#validation
predictions = model.predict(X_val)
rmse = root_mean_squared_error(y_val, predictions)
print(f"validation data RMSE SCORE : {rmse:.4f}")

validation data RMSE SCORE : 8.8401


In [17]:
final_preds = model.predict(X_test)
submission[target_col]= final_preds
submission.to_csv('submission.csv',index=False)